Here we compare the reversal distributions produced by the accumulator model in model_traces_912.ipynb with the empirical reversal distributions. 

In [115]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.patches
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox,Label
import plotly as ply
from dance_sim_tools import trace_objects,utility
from dance_sim_tools.utility import histogram_draw_to_parameters as htp
from dance_sim_tools.utility import draw_reversal_distances_tm
from dance_sim_tools.ipywidget_helpers import slider
import pandas as pd
import dance_sim_tools.trace_objects as trace_objects
import warnings
warnings.filterwarnings('ignore')
import scipy.stats
import matplotlib.gridspec

In [116]:
def rads_to_bls(theta):
    return np.degrees(theta)*(np.pi/9)/2.4
  

In [117]:
#Pull up all of the r_1 and r_0 for each three conditions 

#we collect all the ks and the cs for each of the 3 conditions

def zero_insert(a):
    return np.hstack((np.array([0]),a))

def get_c_hist(m,logN,n_bins):

    n_bins = int(n_bins)
    m = int(m)     #subsample size out of all the flies in the condition
    N = int(np.exp(logN))
    cs = np.full((3,N),np.nan)
 
    for condition in [1,2,3]:

        r_0_filename = 'r_0andr_1/r_0_'+str(condition)+'.txt'    
        empirical_r0s = pd.read_csv(r_0_filename).values.T[0]#, header=None, usecols=[2])

        r_1_filename = 'r_0andr_1/r_1_'+str(condition)+'.txt'    
        empirical_r1s = pd.read_csv(r_1_filename).values.T[0]#, header=None, usecols=[2])

        #For N iterations, draw m pairs of r_1,r_0, and mean(r_1-r_0) using only the subsample
        for i in range(N):
            #randomly select m r_1,r_0 pairs from all r_1s,r_0s for the condition
            #draw m of r_1
            r_1_draw_idx = np.arange(len(empirical_r1s))
            np.random.shuffle(r_1_draw_idx)
            r_1_draw = empirical_r1s[r_1_draw_idx[:m]]
            #Find the r_0 with the same index (=fly index) as those drawn above
            r_0_draw = empirical_r0s[r_1_draw_idx[:m]]

            #Just compute the mean r1-r0 for the subsample
            cs[condition-1,i] = np.mean(r_1_draw-r_0_draw)
             
#             fit_stds[condition-1,i] = np.std(np.degrees(r_1_draw-r_0_draw))


    #Compute and store the histogram of cs for each condition
    cmin,cmax = 0,180
    cbins = np.linspace(cmin,cmax,n_bins)
    cs = np.degrees(cs)    

    
    c_probs = np.zeros((3,len(cbins)-1))
    for i in range(3):
        n,_=np.histogram(cs[i,:],bins=cbins)
        c_probs[i,:]=n/np.sum(n)
    
    return cbins,c_probs
  

def get_k_hist(logN,n_bins):
    n_bins = int(n_bins)
    N = int(np.exp(logN))
    ks = np.full((3,N),np.nan)
 
    for i,condition in enumerate(['1F_1','2F_60_2','2F_90_3']):

        k_filename = 'k_inter_rev/k_inter_'+str(condition)+'.txt'    
        empirical_ks = np.loadtxt(k_filename)#, header=None, usecols=[0,1]).values.T[0]
        #the imported array empirical_ks has shape flies x 30, ( = up to 30 reversal k values)
        fly_count = np.shape(empirical_ks)[0]
        nan_empirical_ks = np.copy(empirical_ks)
        nan_empirical_ks[nan_empirical_ks<0.01] = np.nan
        fly_mean_ks = np.nanmean(nan_empirical_ks,axis=1)
        ks[i,:fly_count] = fly_mean_ks
        
     #Compute and store the histogram of ks for each condition
    kmin,kmax = 0,3.
    kbins = np.linspace(kmin,kmax,n_bins)
    
    k_probs = np.zeros((3,len(kbins)-1))
    for i in range(3):
        n,_=np.histogram(ks[i,:],bins=kbins)
        k_probs[i,:]=n/np.sum(n)
    
    return kbins,k_probs
  
def get_adjusted_emp_r1_hist(n_bins):
    emp_r1s = np.full((3,30),np.nan)
    for i,condition in enumerate(['1','2','3']):

        emp_r1_filename = 'r_0andr_1adjusted/r_1_'+str(condition)+'.txt'    
        empirical_r1s = pd.read_csv(emp_r1_filename).values.T[0]#, header=None, usecols=[2])

        #the imported array empirical_ks has shape flies x 30, ( = up to 30 reversal k values)
        fly_count = np.shape(empirical_r1s)[0]
        emp_r1s[i,:fly_count] = empirical_r1s
        
     #Compute and store the histogram of ks for each condition
    emp_r1_min,emp_r1_max = 40,200.
    emp_r1_bins = np.linspace(emp_r1_min,emp_r1_max,n_bins)
    emp_r1_probs = np.zeros((3,len(emp_r1_bins)-1))
    
    for i in range(3):
        n,_=np.histogram(emp_r1s[i,:],bins=emp_r1_bins)
        emp_r1_probs[i,:]=n/np.sum(n)    
    return emp_r1_bins,emp_r1_probs
    
    
def reversal_count_dist(n_bins):
    reversal_count_bins = np.arange(0,31,1)
    reversal_count_probs = np.zeros((3,30))

    for i,condition in enumerate(['1F_1','2F_60_2','2F_90_3']):

        k_filename = 'k_inter_rev/k_inter_'+str(condition)+'.txt'    
        empirical_ks = np.loadtxt(k_filename)#, header=None, usecols=[0,1]).values.T[0]
        #the imported array empirical_ks has shape flies x 30, ( = up to 30 reversal k values)
        fly_count = np.shape(empirical_ks)[0]
        nan_empirical_ks = np.copy(empirical_ks)
        nan_empirical_ks[nan_empirical_ks<0.01] = np.nan
        reversal_count_vals = np.sum(~np.isnan(nan_empirical_ks),axis=1)
        reversal_count_ns,_ = np.histogram(reversal_count_vals,bins=reversal_count_bins)
        reversal_count_probs[i,:30] = reversal_count_ns/np.sum(reversal_count_ns)
        
        
    #reversal_counts is the same for every condition
    #reversal_count_probs is different for each condition    
    
    return reversal_count_bins,reversal_count_probs

In [134]:
def g(r_0,c_0,c_1,c_2,kappa,logM,
      two_step=True,dt=0.25,condition=0,
      n_bins=40,velocity=10,t_stop=4.,var_type=True,constant_var=0,draw_r0=False):
        
    #Make the trace
    num_segments = 70

    if condition==1:
        food1_loc = 0
        c = c_0
    if condition==2:
        food1_loc = -30
        c = c_1
    if condition==3:
        food1_loc = -45
        c = c_2
    
    food1_loc = np.radians(food1_loc)
    
    n_bins = int(n_bins)
    M = int(np.exp(logM)) #M: number of traces 

    if two_step:
        r_0_est = draw_reversal_distances_tm(M,r_0,velocity,dt,kappa,vdv=var_type,constant_var=constant_var)  #Draw the r_0 values
        c_est = draw_reversal_distances_tm(M,c,velocity,dt,kappa,vdv=var_type,constant_var=constant_var)  #Draw the c values
        r_1 = r_0_est+c_est
    else:
        r_1s = draw_reversal_distances_tm(M,r_0+c,velocity,dt,kappa,vdv=var_type,constant_var=constant_var)  #Draw the r_1 values
        
    reversal_bin_count_array = np.zeros((M,n_bins))
    reversal_count_bins,reversal_count_probs = reversal_count_dist(n_bins)
    r_is = np.full((M,30),np.nan)
    
    
    for i in range(M):
        n_ki_draw = int(np.random.choice(reversal_count_bins[:-1],p=reversal_count_probs[condition-1,:]))
        irds = np.zeros(n_ki_draw)
        irds[0] = r_1[i]
#         if draw_r0:
#             r_0 = draw_emp_r0()
        irds[1:] = draw_reversal_distances_tm(n_ki_draw-1,r_0+c,velocity,dt,kappa)
        r_is[i,0:(n_ki_draw-1)] = irds[1:]
        trace = trace_objects.DistanceListTrace(
                food1_loc,np.radians(irds),np.radians(r_0),t_stop=t_stop,
                    velocity=np.radians(velocity),num_transit_ticks=n_bins)
        reversals_inds,reversals_bin_counts = trace.find_reversal_inds()
        reversal_bin_count_array[i,:] = reversals_bin_counts
        
    transit_tick_interval = 2*np.pi/n_bins  
    bins_model = rads_to_bls(trace.location_bins_rads)
    
    n_model = np.sum(reversal_bin_count_array,axis=0)
    special_ind_bool = (n_model==np.max(n_model)) 
    n_model[special_ind_bool] = 0.
    n_model[special_ind_bool] = np.max(n_model)
    n_model = n_model/(np.sum(n_model))
    bin_width_model=bins_model[1]-bins_model[0]
 
    
    plt.figure(1,figsize=(12,12))#,dpi=200)
    
   
    #New middle plot for the vertical version of the reversal histogram
    ax1 = plt.subplot(221)
    ax1.plot(bins_model[:-1]+bin_width_model/2,n_model,'-o',color='orange')
    ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)

    # Only show ticks on the left and bottom spines
    ax1.yaxis.set_ticks_position('left')
    ax1.xaxis.set_ticks_position('bottom')

    ax1.tick_params(direction='in')
    
    ax1.set_title('Accumulator Model')
    ax1.set_xlim(rads_to_bls(np.array([-np.pi,np.pi])))
    ax1.set_ylabel('Fraction of Reversals')
    ax1.set_xlabel('Location (Body Lengths)')
    
#     ax1.
    
#     plt.ylabel('Fraction of Reversals')
#     xticks = np.arange(-180,210,30)
#     plt.xticks(xticks)#,y_ticklabels,fontsize=15,verticalalignment='bottom')
    
    
  
    #load empirical reversal data
    emp_rev_filename = 'emp_reversal_locs/reversal_locs'+str(condition)+'.txt'
    empirical_reversal_locs = np.loadtxt(emp_rev_filename)

    empirical_reversal_locs =empirical_reversal_locs.flatten()
    empirical_reversal_locs = empirical_reversal_locs[np.nonzero(empirical_reversal_locs)]

    
    n_emp,bins_emp = np.histogram(empirical_reversal_locs, bins=n_bins)#np.linspace(-180,180,25))
    bins_emp = rads_to_bls(bins_emp)
    bin_width = (bins_emp[1]-bins_emp[0])
    n_emp_normd = n_emp/np.sum(n_emp)
    n_emp_normd = n_emp_normd*np.max(n_model)/np.max(n_emp_normd)
    
    ax2 = plt.subplot(222)
    ax2.plot(bins_emp[:-1]+bin_width/2,n_emp_normd,'-o',color='blue')
    ax2.set_title('Empirical Data')
    ax2.set_ylabel('Fraction of Reversals')
    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)

    # Only show ticks on the left and bottom spines
    ax2.yaxis.set_ticks_position('left')
    ax2.xaxis.set_ticks_position('bottom')
    ax2.tick_params(direction='in')
    ax2.set_xlabel('Location (Body Lengths)')
    ax2.set_xlim(rads_to_bls(np.array([-np.pi,np.pi])))
    
    
    

    
    ax3 = plt.subplot(223)
    ax3.plot(bins_emp[:-1]+bin_width/2,n_emp_normd,'-o',color='blue',label='empirical')
    ax3.plot(bins_model[:-1]+bin_width_model/2,n_model,'-o',color='orange',label='model')
    ax3.set_title('Both')
    ax3.legend()
    
    ax3.spines['right'].set_visible(False)
    ax3.spines['top'].set_visible(False)

    # Only show ticks on the left and bottom spines
    ax3.yaxis.set_ticks_position('left')
    ax3.xaxis.set_ticks_position('bottom')
    ax3.tick_params(direction='in')
    ax3.set_xlabel('Location (Body Lengths)')
    
    

    
    if condition==1: #1F
        ax1.axvspan(-1.*bin_width_model/2,bin_width_model/2, facecolor='g', alpha=0.3,label='food') 
        ax2.axvspan(-1.*bin_width_model/2,bin_width_model/2, facecolor='g', alpha=0.3,label='food')      
        ax3.axvspan(-1.*bin_width_model/2,bin_width_model/2, facecolor='g', alpha=0.3,label='food') 
     

        
        
    if condition>1: #2F
        theta_1_loc = -1*rads_to_bls(food1_loc)#/(2*np.pi)
        theta_2_loc = rads_to_bls(food1_loc)#/(2*np.pi)
                
        #in axes coordinates
        ax1.axvspan(theta_1_loc-1.*bin_width_model/2,theta_1_loc+bin_width_model/2, facecolor='g', alpha=0.3,label='food') 
        ax1.axvspan(theta_2_loc-1.*bin_width_model/2,theta_2_loc+bin_width_model/2, facecolor='g', alpha=0.3,label='food') 
        
        ax2.axvspan(theta_1_loc-1.*bin_width_model/2,theta_1_loc+bin_width_model/2, facecolor='g', alpha=0.3,label='food') 
        ax2.axvspan(theta_2_loc-1.*bin_width_model/2,theta_2_loc+bin_width_model/2, facecolor='g', alpha=0.3,label='food') 

        ax3.axvspan(theta_1_loc-1.*bin_width_model/2,theta_1_loc+bin_width_model/2, facecolor='g', alpha=0.3,label='food') 
        ax3.axvspan(theta_2_loc-1.*bin_width_model/2,theta_2_loc+bin_width_model/2, facecolor='g', alpha=0.3,label='food') 


        y_max = np.max(n_emp_normd)
    
#         f1_bar = matplotlib.patches.Rectangle( #note: in axes coordinates
#             (theta_1_loc-(bin_width_model/2)/rads_to_bls(np.pi),-0.05), bin_width_model/rads_to_bls(np.pi),1,
#                 fc="g",transform=ax1.transAxes,clip_on=False,alpha=0.3)
#         f2_bar = matplotlib.patches.Rectangle( #note: in axes coordinates
#             (theta_2_loc-(bin_width_model/2)/rads_to_bls(np.pi),-0.05), bin_width_model/rads_to_bls(np.pi),1,
#                 fc="g",transform=ax1.transAxes,clip_on=False,alpha=0.3)
        
#         ax1.add_artist(f1_bar)
#         ax1.add_artist(f2_bar)

        #       
    
#         theta_1,theta_2 = rads_to_bls(food1_loc),-1*rads_to_bls(food1_loc)
    
#         food_1_min = theta_1-(bin_width_model/2)#/rads_to_bls(np.pi)
#         food_1_max = theta_1+(bin_width_model/2)#/rads_to_bls(np.pi)
        
#         ax1.axvspan(food_1_min,food_1_max,facecolor='g', alpha=0.3,label='food')#,transform=ax1.transAxes) 
#         plt.axvspan(theta_2-(7.5), theta_2+(7.5), facecolor='g', alpha=0.3) 


    case = ['1F','2F_60'][condition-1]
    plt.savefig('reversal_distributions_'+case+'.pdf',format='pdf',dpi=600)
  
    plt.show()
  
      
condition_toggle = widgets.RadioButtons(options=[('1F',1), ('2F 60',2),
                                          ('2F 90',3)],disabled=False,description='condition')     
two_step_toggle  = widgets.RadioButtons(options=[('True',1),('False',0)],disabled=False,description='two_step')
var_type_toggle  = widgets.RadioButtons(options=[('Independent',0),('Value-Dependent',1)],disabled=False,description='var_type')
draw_r0_toggle = widgets.RadioButtons(options=[('fixed',0),('empirical sample',1)],disabled=False,description='draw_r0') 


#start,stop,step,init

sr_0=slider('r_0',20.,80.,5.,40.)
sc_0= slider('c_0',10,180,10,40)
sc_1= slider('c_1',10,180,10,80)
sc_2= slider('c_2',10,180,10,110)
skappa=slider('kappa',0.00,.2,0.01,0.02,readout_format='.2f')
slogM=slider('logM',3,9,1,5)
sdt=slider('dt',0.01,0.5,0.01,0.25,readout_format='.2f')
sn_bins =  slider('n_bins',5,100,1,30)
svelocity=slider('velocity',1,15,1,7)
st_stop = slider('t_stop',0.,5.,.1,3.)
sconstant_var = slider('constant_var',0.,10.,.1,2.5,readout_format='.2f')


sliders = [sr_0,sc_0,sc_1,sc_2,skappa,slogM,sdt,sn_bins,svelocity,st_stop,
           condition_toggle,two_step_toggle,var_type_toggle,sconstant_var,draw_r0_toggle]
           
items = [Box([slider]) for slider in sliders]
  

ui = Box(items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='30%'
))

slider_names = [slider.description for slider in sliders]
param_dict =  dict(zip(slider_names,sliders))


out = widgets.interactive_output(g, param_dict)



display(ui,out)

Box(children=(Box(children=(FloatSlider(value=40.0, continuous_update=False, description='r_0', max=80.0, min=…

Output()

Note: in the top right figure above, we chop off the bin count on the bin containing $f_1-r_0$ so that it is equal to the height of the second highest bin. (because r_0 is always fixed, the first reversal will always fall in the same bin).